In [1]:
# %%cmd
# pip install flask pandas numpy matplotlib scikit-learn jieba zhconv

In [2]:
# %%cmd
# pip list

# 基于朴素贝叶斯的中文文本情感分析

## python 库版本
list
|   Package    | Version |
| :----------: | :-----: |
|    Flask     |  2.0.2  |
|    jieba     | 0.42.1  |
|  matplotlib  |  3.4.3  |
|    numpy     | 1.21.4  |
|    pandas    |  1.3.4  |
| scikit-learn |  1.0.1  |
|    scipy     |  1.7.2  |

## 文本处理
1. 去除停用词，标点符号等
2. 文章参考 https://zhuanlan.zhihu.com/p/39437488
3. 字段含义 https://github.com/SophonPlus/ChineseNlpCorpus/blob/master/datasets/simplifyweibo_4_moods/intro.ipynb

| 字段   | 说明                           |
| :----- | :----------------------------- |
| label  | 0 喜悦，1 愤怒，2 厌恶，3 低落 |
| review | 微博内容                       |

In [3]:
import pandas as pd
import jieba
import zhconv

In [4]:
# 36万多条，带情感标注 新浪微博，包含 4 种情感，其中喜悦约 20 万条，愤怒、厌恶、低落各约 5 万条
emotion = {0: '喜悦', 1: '愤怒', 2: '厌恶', 3: '低落'}
df = pd.read_csv(r'.\data\simplifyweibo_4_moods.csv')
df.sample(20)

,label,review
285610,2,...完全不知道我一直在写错字..【尴尬】左边不是“九”，而是“尤”字少一点；【冒】上面不是...
26604,0,忍不住再推荐一部电影。李秉宪的《甜蜜的人生》，这部电影一定会让你爱上他！瞧，海报多帅啊！
208134,1,对说：喜欢新浪想去应聘想去应聘！呜哈哈
52489,0,吼吼~ 告诉我你们爱谁？！
156050,0,【博友祝福语】第一个母亲节不能在妈妈的身边，妈妈总是嘱咐远方的我要注意身体，最近忙于家里装修...
357201,3,醒了睡睡了醒我怎么好像睡不醒似的啊天爆炸了
93432,0,你的党费好高啊，我半年才交102哦，对不起！各位博友，你的党费交了没？
93267,0,眼看微博就快要破200萬 了....?上次破百萬 時 發 了張 小學 照我好好想想?.......
36744,0,帅百花颁发提名奖携手出席引轰动坤少看着还是那么青葱，造型很像湖北希望小学之行的陈老师
257096,2,我压了我全部家当斯洛伐克赢。。。600点积分都没有了。。。。


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361744 entries, 0 to 361743
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   label   361744 non-null  int64 
 1   review  361744 non-null  object
dtypes: int64(1), object(1)
memory usage: 5.5+ MB


In [6]:
df["label"].value_counts()

0    199496
2     55267
3     55267
1     51714
Name: label, dtype: int64

In [7]:
# 简繁切换
df["review"] = df["review"].apply(lambda x: zhconv.convert(str(x), 'zh-cn'))

In [8]:
from preprocessing.text_split import regex_change, delete_stopwords

In [9]:
# 正则获取主要信息
df["review"] = df["review"].apply(regex_change)
df.sample(20)

,label,review
107253,0,关键时刻动作一致.结果很重要.................
129718,0,。赵薇为江平执导的《康定情歌》祝福视频———苏有朋、居文沛等主演的传奇爱情片《康定情歌》将全...
9099,0,不需要购置机器了~~从原来的保卫人民转化成现在的服务人民。。。
4648,0,不知不觉中小e的活动已经是第期啦~非常感谢e粉们的支持，也希望股市大涨，e粉们中大奖【规则如...
232791,1,哥，你这冷笑话真的不是闹着玩儿的，别这样
124205,0,友友们，早安，每天好心情什么？突击查房？偶还没有任何思想准备！
256613,2,太短了吧指望
232549,1,の，这机器翻译出来的东西，让人晕头转向，头大老师您实在太折磨人了
203069,1,女生下跪？严酷的应试教育，摧残了一届又一届啊！
96635,0,"Suchanamazingdance,thankyou,Keitha."


In [10]:
df["review"].apply(delete_stopwords).sample(20)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\11353\AppData\Local\Temp\jieba.cache
Loading model cost 0.535 seconds.
Prefix dict has been built successfully.


31779                                   奖 股 市 涨 急 死 机 构 散 户
339256            低 端 处 重 发 雷 漫 画 贞 子 发 失 败 感 谢 家 热 情 回 复 雷
310337                                                  钱 钱
80421                               鼻 窍 通 脑 脑 渗 涕 名 副 实 脑 残
142136                                  意 思 目 前 止 觉 爱 放 假 版
98803     o h b a b e 直 播 晚 算 学 习 家 晚 沈 阳 康 熙 盛 典 直 播 址 ...
275300                    醒 继 续 睡 哭 滴 明 天 醒 天 醒 天 醒 天 醒 天 醒
124069                                              发 达 包 养
52805                                       龟 甘 恶 龟 城 猫 猫 走
153536                        尿 憋 配 音 赞 狗 模 样 哥 撒 点 孜 少 放 辣
105587    回 复 知 解 觉 信 天 主 教 貌 似 信 基 督 新 教 解 基 督 教 住 说 帝 ...
26161     噗 H H P 辛 苦 合 体 V 噗 戳 中 H H P 喷 咖 啡 居 萌 娘 噗 H ...
231259                                          天 做 愚 蠢 事 情
316788        偶 家 灾 情 严 重 天 终 明 媚 太 阳 希 暴 雨 山 东 禹 城 防 汛 救 灾
276180                                    节 目 听 急 死 言 姐 救 命
222959                                          姐 腰 肥 圈 救 命
170580    男 士 济 男 士 嘅 福 音 拜 金 男 考 虑 噶 世 

In [16]:
" ".join(jieba.lcut(df.review[2]))

'风格 不 一样 嘛 ， 都 喜欢 ！ 最 喜欢 哪张 ？'

In [28]:
import re
text = "sasdad分割哥哥？阿松大..sd"
patt = re.compile("\w+", re.VERBOSE|re.IGNORECASE)
patt.findall(text)

['sasdad分割哥哥', '阿松大', 'sd']

In [37]:
pat = re.compile("[\u4E00-\u9FA5]+", re.VERBOSE|re.IGNORECASE)
s = pat.findall(df.review[339090])
s

['姐姐提醒了我',
 '冥冥之中觉得',
 '画皮',
 '花木兰',
 '锦衣卫',
 '是纠葛在一起前世今生的恩怨情仇',
 '前生佩蓉欠了勇哥的情',
 '来世变作大漠中痴痴等候青龙的乔花',
 '而前世王生欠了佩蓉的债',
 '今生他做了木兰背后的文泰']

In [39]:
tl = []
for i in s:
    tl += [x for x in jieba.cut(i)]
" ".join(tl)

'姐姐 提醒 了 我 冥冥之中 觉得 画皮 花木兰 锦衣卫 是 纠葛 在 一起 前世 今生 的 恩怨 情仇 前生 佩蓉 欠 了 勇哥 的 情 来世 变 作 大漠 中 痴痴 等候 青龙 的 乔花 而 前世 王生 欠 了 佩蓉 的 债 今生 他 做 了 木兰 背后 的 文泰'

In [54]:
with open(".\data\stopwords.dat", encoding="utf-8") as fp:
    lines = fp.readlines()
stop_word = []
for line in lines:
    stop_word.append(line.rsplit("\n")[0])
with open("stopwords.txt", "w", encoding="utf-8") as fp:
    fp.write(" ".join(stop_word))


In [55]:
[word for word in jieba.lcut(df.review[26161]) if word not in stop_word]

['噗',
 '哈哈哈哈',
 'HHP',
 '辛苦',
 '合体',
 'V5',
 '噗',
 '哈哈哈',
 '戳',
 '中',
 'HHP',
 '喷',
 '咖啡',
 '萌',
 '娘',
 '哈哈哈',
 '哈哈哈',
 '噗',
 'HHP',
 '草泥马',
 '法可',
 '鱿',
 '合体',
 '魅力',
 '不容忽视',
 '噗',
 '....']

In [58]:
def jieba_cut_text(text, stop_word, cut_by=" "):
    patt = re.compile("[\u4E00-\u9FA5]+", re.VERBOSE|re.IGNORECASE)
    word_list = []
    for t in patt.findall(text):
        word_list += [word for word in jieba.cut(t) if word not in stop_word]

    return cut_by.join(word_list)

jieba_cut_text(df.review[339090], stop_word)

'姐姐 提醒 冥冥之中 画皮 花木兰 锦衣卫 纠葛 前世 今生 恩怨 情仇 前生 佩蓉 欠 勇哥 情 来世 变 作 大漠 中 痴痴 等候 青龙 乔花 前世 王生 欠 佩蓉 债 今生 做 木兰 背后 文泰'